In [3]:
!pip install datasets peft transformers


  Using cached peft-0.12.0-py3-none-any.whl.metadata (13 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 k

In [4]:
import os
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
import pandas as pd
from datasets import Dataset, DatasetDict

In [5]:
# Load the CSV dataset
df = pd.read_csv('/content/Siber_Guvenlik_Terim_Karsiliklari_duzenlenmis.csv', delimiter=';')

FileNotFoundError: [Errno 2] No such file or directory: '/content/Siber_Guvenlik_Terim_Karsiliklari_duzenlenmis.csv'

In [ ]:
# Convert to Hugging Face dataset format
dataset = Dataset.from_pandas(df[['English Term', 'Turkish Term']])

In [ ]:
# Split dataset into training and validation
split_dataset = dataset.train_test_split(test_size=0.1)
tokenized_datasets = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

In [ ]:
# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')

In [ ]:
def preprocess_function(examples):
    inputs = [f"translate English to Turkish: {ex}" for ex in examples['English Term']]
    targets = examples['Turkish Term']

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length').input_ids

    # Replace padding token id with -100 in labels
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_seq] for labels_seq in labels]

    model_inputs["labels"] = labels
    return model_inputs

In [6]:
# Apply preprocessing
tokenized_datasets = tokenized_datasets.map(preprocess_function, batched=True)

NameError: name 'tokenized_datasets' is not defined

In [ ]:
# Define training arguments with mixed precision disabled
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',               # Directory to save the model and results
    evaluation_strategy='epoch',          # Evaluate after every epoch
    learning_rate=2e-5,                   # Learning rate
    per_device_train_batch_size=2,        # Batch size for training
    per_device_eval_batch_size=2,         # Batch size for evaluation
    num_train_epochs=3,                   # Number of epochs
    weight_decay=0.01,                    # Weight decay
    save_total_limit=1,                   # Keep only the latest checkpoint
    logging_dir='./logs',                 # Directory for logs
    logging_steps=10,                     # Log every 10 steps
    fp16=True,                           # Ensure mixed precision is disabled
)

In [ ]:
# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)

In [ ]:
# Start training
trainer.train()

In [ ]:
# Save the fine-tuned model and tokenizer
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')

In [ ]:
# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./fine-tuned-model')
tokenizer = T5Tokenizer.from_pretrained('./fine-tuned-model')

You are using a model of type mbart to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.
Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at facebook/mbart-large-50-many-to-many-mmt and are newly initialized: ['decoder.block.0.layer.0.SelfAttention.k.weight', 'decoder.block.0.layer.0.SelfAttention.o.weight', 'decoder.block.0.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight', 'decoder.block.0.layer.0.SelfAttention.v.weight', 'decoder.block.0.layer.0.layer_norm.weight', 'decoder.block.0.layer.1.EncDecAttention.k.weight', 'decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.0.layer.1.EncDecAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.v.weight', 'decoder.block.0.layer.1.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'decoder.block.0

ValueError: Target modules {'q_proj', 'v_proj'} not found in the base model. Please check the target modules and try again.

In [ ]:
# Load the fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./fine-tuned-model')
tokenizer = T5Tokenizer.from_pretrained('./fine-tuned-model')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Der türkische Mißbrauch legitimer Dienste hat offensichtliche Vorteile für Angreifer in der Tatsache, dass sie nicht nur eine kostengünstige Lösung sind, sondern vor allem bieten sie eine heimliche Kommunikation innerhalb von Netzwerken, da sie wahrscheinlich nicht durch Sicherheitsprodukte oder Firewalls geblockt werden.


In [ ]:
def translate_text_directly(text, model, tokenizer, max_length=128):
    # Format the input text
    input_text = f"translate English to Turkish: {text}"
    inputs = tokenizer(input_text, return_tensors='pt', max_length=max_length, truncation=True)

    # Generate the translation
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,  # Beam search for better quality
            early_stopping=True
        )

    # Decode the generated text
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Example usage
text_to_translate = "Abusing legitimate services has obvious benefits for attackers in that they're not only a low-cost solution, but more importantly, they offer a clandestine way of communication inside networks, as they are unlikely to be blocked by security products or firewalls."
translated_text = translate_text_directly(text_to_translate, model, tokenizer)
print(translated_text)

# en son

In [ ]:
import os
import torch
from transformers import MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
import pandas as pd
from datasets import Dataset, DatasetDict

df = pd.read_csv('/content/Siber_Guvenlik_Terim_Karsiliklari_duzenlenmis.csv', delimiter=';')

dataset = Dataset.from_pandas(df[['English Term', 'Turkish Term']])

split_dataset = dataset.train_test_split(test_size=0.1)
tokenized_datasets = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

model_name = 'facebook/mbart-large-50-many-to-many-mmt'
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

def preprocess_function(examples):
    inputs = examples['English Term']
    targets = examples['Turkish Term']

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length').input_ids

    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_seq] for labels_seq in labels]

    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = tokenized_datasets.map(preprocess_function, batched=True)

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',               # Directory to save the model and results
    evaluation_strategy='epoch',          # Evaluate after every epoch
    learning_rate=2e-5,                   # Learning rate
    per_device_train_batch_size=2,        # Batch size for training
    per_device_eval_batch_size=2,         # Batch size for evaluation
    num_train_epochs=3,                   # Number of epochs
    weight_decay=0.01,                    # Weight decay
    save_total_limit=1,                   # Keep only the latest checkpoint
    logging_dir='./logs',                 # Directory for logs
    logging_steps=10,                     # Log every 10 steps
    fp16=True,                           # Mixed precision training
)


trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)


trainer.train()


model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


Map:   0%|          | 0/3554 [00:00<?, ? examples/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.461800,1.138709
2,0.597800,1.044733
3,0.130200,1.061477


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/sentencepiece.bpe.model',
 './fine-tuned-model/added_tokens.json')

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 1.243155598640442, 'eval_runtime': 7.6508, 'eval_samples_per_second': 51.629, 'eval_steps_per_second': 25.88, 'epoch': 3.0}


In [ ]:
from transformers import MBartForConditionalGeneration, MBartTokenizer
import torch

model = MBartForConditionalGeneration.from_pretrained('./fine-tuned-model')
tokenizer = MBartTokenizer.from_pretrained('./fine-tuned-model')

def translate_text(text, model, tokenizer, max_length=128):

    inputs = tokenizer(text, return_tensors='pt', max_length=max_length, truncation=True)


    inputs['decoder_input_ids'] = tokenizer.encode('tr_XX', return_tensors='pt')


    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )


    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text


text_to_translate = "NCSC Assessment (NCSC-A) is the authoritative voice on the cyber threat to the UK. We fuse all-source information – classified intelligence, industry knowledge, academic material and open source – to provide independent key judgements that inform policy decision making and improve UK cyber security. We work closely with government, industry and international partners for expert input into our assessments."
translated_text = translate_text(text_to_translate, model, tokenizer)
print(translated_text)


tr_XX Yükleşik kaynak bilgilerini birleştirmek ( sınıflandırılmış istihbarat, sanayi bilgisi, akademik malzeme ve açık kaynak) politika karar verme ve Yükleşik siber güvenliği geliştirmek için bağımsız temel karar verme bildirimleri sağlama için tüm kaynak bilgilerini birleştirmek ( sınıflandırılmış istihbarat, sanayi bilgisi, akademik malzeme ve açık kaynak) uzman katkıları için hükümet, sanayi ve uluslararası ortaklarla yakın çalışmaq


# optimized

In [ ]:
!pip install transformers datasets peft


In [ ]:
import os
import torch
from transformers import MBartForConditionalGeneration, MBartTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import Dataset, DatasetDict
from peft import LoraConfig, get_peft_model
import pandas as pd

# Load the CSV dataset
df = pd.read_csv('/content/Siber_Guvenlik_Terim_Karsiliklari_duzenlenmis.csv', delimiter=';')

# Convert to Hugging Face dataset format
dataset = Dataset.from_pandas(df[['English Term', 'Turkish Term']])

# Split dataset into training and validation
split_dataset = dataset.train_test_split(test_size=0.1)
tokenized_datasets = DatasetDict({
    'train': split_dataset['train'],
    'validation': split_dataset['test']
})

# Load tokenizer and model
model_name = 'facebook/mbart-large-50-many-to-many-mmt'
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

# Apply LoRA for efficient fine-tuning
lora_config = LoraConfig(
    r=8,                           # Rank of the update matrices
    lora_alpha=32,                 # Scaling factor for the updates
    target_modules=["q_proj", "v_proj"],  # Target attention layers
    lora_dropout=0.1,              # Dropout probability for LoRA layers
    bias="none",                   # Bias setting for LoRA layers
    task_type="SEQ_2_SEQ_LM"       # Task type
)
model = get_peft_model(model, lora_config)

def preprocess_function(examples):
    inputs = examples['English Term']
    targets = examples['Turkish Term']

    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length').input_ids

    # Replace padding token id with -100 in labels
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_seq] for labels_seq in labels]

    model_inputs["labels"] = labels
    return model_inputs

# Apply preprocessing
tokenized_datasets = tokenized_datasets.map(preprocess_function, batched=True)

# Define optimized training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',               # Directory to save the model and results
    evaluation_strategy='epoch',          # Evaluate after every epoch
    learning_rate=3e-5,                   # Slightly increased learning rate for faster convergence
    per_device_train_batch_size=4,        # Increased batch size for T4 High RAM
    per_device_eval_batch_size=4,         # Increased batch size for evaluation
    num_train_epochs=5,                   # Increased number of epochs for better performance
    weight_decay=0.01,                    # Weight decay to prevent overfitting
    save_total_limit=2,                   # Keep only the latest two checkpoints
    logging_dir='./logs',                 # Directory for logs
    logging_steps=10,                     # Log every 10 steps
    fp16=True,                           # Mixed precision training for faster computation
    gradient_accumulation_steps=2,        # Accumulate gradients to simulate a larger batch size
    lr_scheduler_type='cosine',           # Use cosine learning rate schedule
    warmup_steps=500,                     # Number of steps to warm up learning rate
)

# Initialize the Trainer with LoRA
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer,
)

# Start training
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('./fine-tuned-model')
tokenizer.save_pretrained('./fine-tuned-model')



The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


Map:   0%|          | 0/3554 [00:00<?, ? examples/s]

Map:   0%|          | 0/395 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,3.159300,2.615650
2,2.077400,1.898808
4,1.779200,1.853537


('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/sentencepiece.bpe.model',
 './fine-tuned-model/added_tokens.json')

In [ ]:
def translate_text(text, model, tokenizer, max_length=128):
    # Prepare the text for translation
    source_lang = 'en_XX'
    target_lang = 'tr_XX'

    # Tokenize the input text
    inputs = tokenizer(f'{source_lang} {text}', return_tensors='pt', max_length=max_length, truncation=True, padding='max_length')

    # Generate translation with the model
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            decoder_start_token_id=tokenizer.lang_code_to_id[target_lang],  # Set the target language code
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

    # Decode and return the translated text
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text


In [ ]:
text_to_translate = "Clickjacking is a common style of phishing."
translated_text = translate_text(text_to_translate, model, tokenizer)
print(translated_text)


KeyError: 'tr_XX'

In [ ]:
def translate_text(text, model, tokenizer, max_length=128):
    # Define source and target languages
    source_lang = 'en_XX'
    target_lang = 'tr_TR'  # Use the correct Turkish language code

    # Tokenize input with the source language token
    input_text = f'{source_lang} {text}'
    inputs = tokenizer(input_text, return_tensors='pt', max_length=max_length, truncation=True)

    # Generate translation
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            decoder_start_token_id=tokenizer.lang_code_to_id[target_lang],  # Set target language code
            max_length=max_length,
            num_beams=4,
            early_stopping=True
        )

    # Decode and return the translated text
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text


In [ ]:
text_to_translate = "Clickjacking is a common style of phishing."
translated_text = translate_text(text_to_translate, model, tokenizer)
print(translated_text)


ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya ya
